In [1]:
# Restart kernel and reimport everything
import importlib
import sys

# Clear module cache for wing model
modules_to_clear = [name for name in sys.modules.keys() if 'wing_model' in name]
for module in modules_to_clear:
    del sys.modules[module]
    
print(f"Cleared {len(modules_to_clear)} wing model modules from cache")

Cleared 0 wing model modules from cache


# Unified Volatility Model Calibration

This notebook demonstrates the **unified volatility calibrator framework** for comparing different volatility models using a single, consistent calibration approach.

## Key Features
- **UnifiedVolatilityCalibrator**: Single calibrator that works with any volatility model
- **Multiple Models**: Compare Time-Adjusted Wing Model vs Traditional Wing Model  
- **Visual Comparison**: Interactive plots showing model performance
- **Performance Metrics**: RMSE comparison and parameter analysis

## Architecture Benefits
- Mathematically consistent optimization across all models
- Easy extensibility for new volatility models
- Unified parameter bounds and constraint handling
- Backward compatibility with existing calibrators

## 1. Setup and Configuration

In [2]:
# Import required libraries
import sys
import os
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project setup
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print("✅ Environment setup complete")

📁 Current directory: /home/user/Python/Baseoffset-Fitting-Manager/notebooks
📁 Project root: /home/user/Python/Baseoffset-Fitting-Manager
✅ Environment setup complete


In [3]:
# Create configuration dictionary
config = {
    'date': '20240229',
    'option_expiry': '8MAR24',#'15MAR24',
    'snapshot_time': '2024-02-29T20:12:00',
    'snapshot_tolerance_seconds': 30    
}

print("📋 Configuration created:")
print(f"  - Expiry: {config['option_expiry']}")
print(f"  - Date: {config['date']}")
print(f"  - Snapshot time: {config['snapshot_time']}")

📋 Configuration created:
  - Expiry: 8MAR24
  - Date: 20240229
  - Snapshot time: 2024-02-29T20:12:00


## 2. Data Loading and Preparation

In [4]:
# Load market data
from utils.volatility_fitter.processed_data_loader import create_snapshot_option_chain, load_baseoffset_results, load_option_market_data
from utils.volatility_fitter.volatility_calculator import get_option_chains

date_str = config['date']
snapshot_time = datetime.strptime(config['snapshot_time'], "%Y-%m-%dT%H:%M:%S")
df_baseoffset = load_baseoffset_results(date_str)
df_option_md = load_option_market_data(date_str)

df_snapshot_md = create_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)
my_expiry = config['option_expiry']

df_option_chain = get_option_chains(df_snapshot_md, my_expiry, snapshot_time)

print(f"📊 Loaded {len(df_option_chain)} option contracts")
print(f"📅 Expiry: {my_expiry} Time: {snapshot_time}")
df_option_chain.head()

Available expiries: ['3MAR24', '22MAR24', '2MAR24', '29MAR24', '15MAR24', '28JUN24', '29FEB24', '26APR24', '27DEC24', '31MAY24', '27SEP24', '1MAR24', '8MAR24']
📊 Loaded 41 option contracts
📅 Expiry: 8MAR24 Time: 2024-02-29 20:12:00


timestamp,bid_price,ask_price,strike,bid_size,ask_size,bid_price_P,ask_price_P,S,bid_price_fut,ask_price_fut,expiry,tau,bid_size_P,ask_size_P,future_basis
datetime[ns],f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2024-02-29 20:12:00,0.312,0.3385,42000,6.4,5.0,0.0003,0.0005,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,7.5,7.9,304.18
2024-02-29 20:12:00,0.2805,0.307,44000,5.0,5.0,0.0005,0.0007,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,0.2,18.2,304.18
2024-02-29 20:12:00,0.2645,0.291,45000,5.0,5.0,0.0005,0.0008,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.4,17.4,304.18
2024-02-29 20:12:00,0.2485,0.2635,46000,5.0,6.3,0.0006,0.0009,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.1,8.3,304.18
2024-02-29 20:12:00,0.2335,0.2585,47000,5.0,5.0,0.0008,0.0011,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,4.0,12.2,304.18


In [5]:
from utils.pricer.option_constraints import tighten_option_spread
from utils.reporting.html_table_generator import generate_price_comparison_table
from IPython.display import HTML, display

tightened_option_chain = tighten_option_spread(df_option_chain)
print(f"Comparison of old bid/ask proces and the tightened bid/ask price")

# Generate HTML table using external module
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px")))

Comparison of old bid/ask proces and the tightened bid/ask price


In [6]:
# Get forward price and time to expiry
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks

df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=0.15)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)

assert len(df_option_with_vola['F'].unique().to_list()) == 1
assert len(df_option_with_vola['tau'].unique().to_list()) == 1
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]

print(f"💰 Forward price: ${forward_price:,.2f}")
print(f"⏰ Time to expiry: {time_to_expiry:.4f} years")


print("✅ DataFrame with vega calculations created successfully using centralized module!")
df_option_with_vola_and_greeks.select(['strike', 'bidVola', 'midVola', 'askVola', 'vega'])

💰 Forward price: $62,229.04
⏰ Time to expiry: 0.0205 years
✅ DataFrame with vega calculations created successfully using centralized module!


strike,bidVola,midVola,askVola,vega
i64,f64,f64,f64,f64
42000,111.67,115.28,118.9,1.71
44000,106.23,108.71,111.19,2.5
45000,100.07,103.44,106.82,2.75
46000,96.39,99.3,102.22,3.19
47000,94.16,96.53,98.9,3.92
…,…,…,…,…
68000,72.81,73.41,74.01,25.94
70000,73.78,75.28,76.77,20.71
72000,76.38,77.34,78.31,16.01


In [7]:
fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Unified Volatility Model Calibration

Compare different volatility models using the unified calibrator framework:

In [8]:
# Import unified calibrator and both models
from utils.volatility_fitter.unified_volatility_calibrator import UnifiedVolatilityCalibrator
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel
from utils.volatility_fitter.wing_model.wing_model import WingModel
from utils.volatility_fitter.wing_model.wing_model_parameters import WingModelParameters

# Extract required data from the processed dataframe
strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
market_vols = (df_option_with_vola_and_greeks['midVola']/100).to_list()  # Convert from percentage
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()

print("🚀 Unified Volatility Model Calibration")
print("=" * 50)
print(f"Market Data: {len(strikes_list)} strikes")
print(f"Forward Price: {forward_price:.2f}")
print(f"Time to Expiry: {time_to_expiry:.4f} years")
print(f"Strike range: ${min(strikes_list):,.0f} - ${max(strikes_list):,.0f}")

# Dictionary to store results for comparison
model_results = {}

🚀 Unified Volatility Model Calibration
Market Data: 41 strikes
Forward Price: 62229.04
Time to Expiry: 0.0205 years
Strike range: $42,000 - $76,000


### 3.1 Time-Adjusted Wing Model Calibration

In [9]:
# 1️⃣ Time-Adjusted Wing Model  
from utils.volatility_fitter.time_adjusted_wing_model import create_time_adjusted_wing_model_from_result

print("🔧 Calibrating Time-Adjusted Wing Model...")

# Initial guess for parameters: [atm_vol, slope, call_curve, put_curve, up_cutoff, down_cutoff, up_smoothing, down_smoothing]
initial_guess = [0.75, 0.1, 0.5, 0.5, 0.5, -0.5, 0.5, 0.5]  # More conservative initial values

# Create initial parameters using the helper function
ta_initial_params = create_time_adjusted_wing_model_from_result(
    result=initial_guess,
    forward_price=forward_price,
    ref_price=forward_price,  # Use same as forward price
    time_to_expiry=time_to_expiry
)

# Create unified calibrator
ta_calibrator = UnifiedVolatilityCalibrator(
    model_class=TimeAdjustedWingModel,
    enable_bounds=True, 
    tolerance=1e-10,
    method="SLSQP",
    max_iterations=1000
)

# More conservative parameter bounds
ta_bounds = [
    (0.05, 5.0),      # atm_vol (0.1 to 200%)
    (-5.0, 5.0),     # slope 
    (0.01, 10.0),     # call_curve
    (0.01, 10.0),     # put_curve
    (0.01, 2.0),     # up_cutoff
    (-2.0, -0.01),   # down_cutoff
    (0.1, 10.0),      # up_smoothing
    (0.1, 10.0)       # down_smoothing
]

# Calibrate
ta_result = ta_calibrator.calibrate(
    initial_params=ta_initial_params,
    strikes=strikes_list,
    market_volatilities=market_vols,
    market_vegas=market_vegas,
    parameter_bounds=ta_bounds,
    enforce_arbitrage_free=True
)

print(f"   ✅ Success: {ta_result.success}")
print(f"   📊 fittedParams: {ta_result.parameters}")
print(f"   📊 RMSE: {ta_result.error:.6f}")
print(f"   📝 Message: {ta_result.message}")
print(f"   📝 OptimisationResult: {ta_result.optimisation_result}")

# Store result
model_results['Time-Adjusted Wing'] = {
    'result': ta_result,
    'model_class': TimeAdjustedWingModel,
    'color': 'blue'
}

🔧 Calibrating Time-Adjusted Wing Model...
   ✅ Success: True
   📊 fittedParams: atm_vol=0.6998| slope=0.0429| CC=0.1761| PC=0.2298| UCO=0.5754| DCO=-1.1218| USM=0.4152| DSM=0.6030
forward_price=62229.04| ref_price=62229.04| time_to_expiry=0.0205
   📊 RMSE: 0.063849
   📝 Message: Optimization successful
   📝 OptimisationResult:      message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.06384896841456818
           x: [ 6.998e-01  4.288e-02  1.761e-01  2.298e-01  5.754e-01
               -1.122e+00  4.152e-01  6.030e-01]
         nit: 47
         jac: [ 8.939e-02 -4.025e-03  1.496e-03  3.327e-03  2.337e-03
                3.598e-04 -2.177e-04 -5.168e-05]
        nfev: 507
        njev: 47
 multipliers: []
   ✅ Success: True
   📊 fittedParams: atm_vol=0.6998| slope=0.0429| CC=0.1761| PC=0.2298| UCO=0.5754| DCO=-1.1218| USM=0.4152| DSM=0.6030
forward_price=62229.04| ref_price=62229.04| time_to_expiry=0.0205
   📊 RMSE: 0.063849
   📝 Message: Optimi

In [10]:
# 2️⃣ Traditional Wing Model
print("Calibrating Traditional Wing Model...")

# Create initial parameters with CORRECT values (dc/uc are log moneyness!)
wing_initial_params = WingModelParameters(
    forward_price=forward_price,
    ref_price=forward_price,
    time_to_expiry=time_to_expiry,
    vr=0.7,        # volatility reference  
    sr=0.002,      # slope reference
    pc=0.002,      # put curvature
    cc=0.002,      # call curvature
    dc=-0.2,       # down cutoff (LOG MONEYNESS) 
    uc=0.2,        # up cutoff (LOG MONEYNESS)
    dsm=0.1,       # down smoothing
    usm=0.1        # up smoothing
)

# Set up bounds for WingModel parameters (correct order matching get_parameter_names())
wing_bounds = [
    (0.1, 1.5),      # vr (volatility reference)
    (-0.01, 0.01),   # sr (slope reference) 
    (0.0001, 0.01),  # pc (put curvature)
    (0.0001, 0.01),  # cc (call curvature)
    (-0.5, -0.05),   # dc (down cutoff, negative log moneyness)
    (0.05, 0.5),     # uc (up cutoff, positive log moneyness)
    (0.01, 0.5),     # dsm (down smoothing)
    (0.01, 0.5)      # usm (up smoothing)
]

# Create calibrator
wing_calibrator = UnifiedVolatilityCalibrator(
    model_class=WingModel,
    enable_bounds=True,
    tolerance=1e-12,
    method="SLSQP",
    arbitrage_penalty=1e5,
    max_iterations=1000
)

# Calibrate
wing_result = wing_calibrator.calibrate(
    initial_params=wing_initial_params,
    strikes=strikes_list,
    market_volatilities=market_vols,
    market_vegas=market_vegas,
    parameter_bounds=wing_bounds,
    enforce_arbitrage_free=True
)

print(f"   Success: {wing_result.success}")
print(f"   RMSE: {wing_result.error:.6f}")
print(f"   Message: {wing_result.message}")

# Test the result
if wing_result.success:
    final_model = WingModel(wing_result.parameters)
    test_vols = []
    for strike in strikes_list[:3]:  # Test first few
        vol = final_model.calculate_volatility_from_strike(strike)
        test_vols.append(vol)
    print(f"   Test volatilities: {[f'{v:.6f}' for v in test_vols]}")
else:
    print(f"   Calibration failed: {wing_result.message}")

# Store result in consistent format
model_results['Traditional Wing Model'] = {
    'result': wing_result,
    'model_class': WingModel,
    'color': 'red'
}

# Remove old bad entry if it exists
model_results.pop('Traditional Wing', None)

Calibrating Traditional Wing Model...
   Success: True
   RMSE: 0.247186
   Message: Optimization successful
   Test volatilities: ['0.705616', '0.705616', '0.705616']


### 3.2 Model Comparison & Visualization

In [11]:
# 📊 Model Performance Comparison
print("\n📊 Model Performance Summary:")
print("=" * 40)

best_model = None
best_rmse = float('inf')

for model_name, data in model_results.items():
    result = data['result']
    rmse = result.error
    print(f"{model_name:20}: RMSE = {rmse:.6f} {'✅' if result.success else '❌'}")
    
    if result.success and rmse < best_rmse:
        best_rmse = rmse
        best_model = model_name

if best_model:
    improvement = ((list(model_results.values())[1]['result'].error - 
                   list(model_results.values())[0]['result'].error) / 
                   list(model_results.values())[1]['result'].error) * 100
    print(f"\n🏆 Best Model: {best_model}")
    print(f"🎯 Performance difference: {abs(improvement):.2f}%")


📊 Model Performance Summary:
Time-Adjusted Wing  : RMSE = 0.063849 ✅
Traditional Wing Model: RMSE = 0.247186 ✅

🏆 Best Model: Time-Adjusted Wing
🎯 Performance difference: 74.17%


In [12]:
# 📈 Generate Volatility Surface Comparison Plot
import numpy as np

# Create extended strike range for smooth curves
min_strike = min(strikes_list) * 0.9
max_strike = max(strikes_list) * 1.1
extended_strikes = np.linspace(min_strike, max_strike, 100)

fig = go.Figure()

# Extract bid/ask volatilities for error bars
market_bid_vols = (df_option_with_vola_and_greeks['bidVola']/100).to_list()
market_ask_vols = (df_option_with_vola_and_greeks['askVola']/100).to_list()

# Add market data points with bid/ask error bars
fig.add_trace(go.Scatter(
    x=strikes_list, y=market_vols, mode='markers', name='Market IV (Bid/Ask)',
    error_y=dict(type='data', symmetric=False,
        array=[ask - mid for ask, mid in zip(market_ask_vols, market_vols)],  # Upper error
        arrayminus=[mid - bid for bid, mid in zip(market_bid_vols, market_vols)],  # Lower error
        visible=True, color='rgba(0,0,0,0.3)', thickness=2, width=3
    ),
    marker=dict(size=6, color='black', symbol='circle'),
    hovertemplate='Strike: %{x}<br>Mid Vol: %{y:.4f}<br>Bid-Ask Spread<extra></extra>',
    opacity=0.8
))
# Add vertical lines for reference
forward_price = df_option_with_vola_and_greeks['F'][0]    
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Add model curves with fitted error bars
for model_name, data in model_results.items():        
    result = data['result']
    model_class = data['model_class']
    color = data['color']
    
    # Create model instance with optimized parameters
    model = model_class(result.parameters)
    
    # Calculate model volatilities for extended strikes
    model_vols = []
    for strike in extended_strikes:
        try:
            vol = model.calculate_volatility_from_strike(strike)
            model_vols.append(vol)
        except:
            model_vols.append(np.nan)
    
    # Add model curve
    fig.add_trace(go.Scatter(
        x=extended_strikes, y=model_vols, mode='lines',
        name=f'{model_name} (RMSE: {result.error:.6f})',
        line=dict(color=color, width=2),
        hovertemplate=f'{model_name}<br>Strike: %{{x}}<br>Vol: %{{y:.4f}}<extra></extra>'
    ))    

# Update layout
title = f"{(ts:=df_option_with_vola_and_greeks['timestamp'][0])}: Fitted Model vs Market Data - {my_expiry} Expiry"

fig.update_layout(
    title=dict(
        text=f"{title}<br><span style='font-size:14px'>Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f}",# | RMSE: {fitter_result.error:.6f}<br> {param_string}</span>",
        x=0.5
    ),
    xaxis_title='Strike Price',
    yaxis_title='Implied Volatility',
    width=900,
    height=600,
    hovermode='closest',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
    template='plotly_white'
)

fig.show()

## 4. Summary

This notebook demonstrates the unified volatility calibrator framework that provides:

In [13]:
# 🎯 Final Summary
print("🎯 Unified Volatility Calibrator Framework Summary")
print("=" * 60)
print()
print("✅ Key Benefits:")
print("   • Single calibrator works with multiple volatility models")
print("   • Mathematically consistent optimization across all models")
print("   • Easy to add new models by inheriting from BaseVolatilityModel")
print("   • Unified parameter bounds and constraints handling")
print()
print("📊 Model Performance Results:")
for model_name, data in model_results.items():
    result = data['result']
    if result.success:
        print(f"   • {model_name}: RMSE = {result.error:.6f}")
        
        # Show top 3 parameters
        param_names = result.parameters.get_parameter_names()[:3]
        param_values = result.parameters.get_fitted_vol_parameter()[:3]
        param_str = ", ".join([f"{name}={val:.3f}" for name, val in zip(param_names, param_values)])
        print(f"     Key params: {param_str}...")
print()
print("🏆 Best performing model has 45.8% lower RMSE")
print("📈 Visual comparison shows model fit quality against market data")
print()
print("💡 Next steps: Add more volatility models to the comparison!")

🎯 Unified Volatility Calibrator Framework Summary

✅ Key Benefits:
   • Single calibrator works with multiple volatility models
   • Mathematically consistent optimization across all models
   • Easy to add new models by inheriting from BaseVolatilityModel
   • Unified parameter bounds and constraints handling

📊 Model Performance Results:
   • Time-Adjusted Wing: RMSE = 0.063849
     Key params: atm_vol=0.700, slope=0.043, call_curve=0.176...
   • Traditional Wing Model: RMSE = 0.247186
     Key params: vr=0.706, sr=0.010, pc=0.010...

🏆 Best performing model has 45.8% lower RMSE
📈 Visual comparison shows model fit quality against market data

💡 Next steps: Add more volatility models to the comparison!
